# Convolutional Neural Network in Keras

Bulding a Convolutional Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [1]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [2]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Layer, Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, LeakyReLU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
# from keras_contrib.layers.advanced_activations import SineReLU

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Load data

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [4]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_test = X_test.astype("float32")/255.

In [5]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### SineReLU V2

In [6]:
class SineReLU(Layer):

    def __init__(self, epsilon=0.0055, **kwargs):
        super(SineReLU, self).__init__(**kwargs)
        self.supports_masking = True
        self.epsilon = K.cast_to_floatx(epsilon)

    def call(self, Z):
        m = self.epsilon * (K.sin(Z) - K.cos(Z))        
        A = K.maximum(m, Z)
        return A

    def get_config(self):
        config = {'epsilon': float(self.epsilon)}
        base_config = super(SineReLU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

get_custom_objects().update({'SineReLU': SineReLU})

#### Design Neural Network architecture

In [7]:
epsilon_cnn, epsilon_dense = 10**-5, 10**-5 #0.025, 0.006

model = Sequential()

model.add(Conv2D(32, 7, padding = 'same', input_shape = (28, 28, 1)))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))

model.add(Conv2D(32, 7, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, 3, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))

model.add(Conv2D(64, 3, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.30))

model.add(Conv2D(128, 2, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))

model.add(Conv2D(128, 2, padding = 'same'))
# model.add(SineReLU(epsilon_cnn))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(512))
# model.add(SineReLU(epsilon_dense))
# model.add(LeakyReLU(alpha=0.01))
model.add(Activation('relu'))
model.add(Dropout(0.50))

model.add(Dense(10, activation = "softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        1600      
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        50208     
_________________________________________________________________
activation_2 (Activation)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        18496     
__________

#### Callbacks

In [8]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-cnn-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')

tensorboard = TensorBoard("logs/convnet-mnist-relu-IV")

#### Configure model

In [9]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [10]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 5, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

Train on 54000 samples, validate on 6000 samples
Epoch 1/5
54000/54000 [==============================] - 314s 6ms/step - loss: 0.3413 - acc: 0.8863 - val_loss: 0.0579 - val_acc: 0.9833
Epoch 2/5
54000/54000 [==============================] - 308s 6ms/step - loss: 0.0858 - acc: 0.9742 - val_loss: 0.0470 - val_acc: 0.9857
Epoch 3/5
54000/54000 [==============================] - 310s 6ms/step - loss: 0.0644 - acc: 0.9809 - val_loss: 0.0307 - val_acc: 0.9902
Epoch 4/5
54000/54000 [==============================] - 317s 6ms/step - loss: 0.0523 - acc: 0.9839 - val_loss: 0.0290 - val_acc: 0.9910
Epoch 5/5
54000/54000 [==============================] - 315s 6ms/step - loss: 0.0438 - acc: 0.9866 - val_loss: 0.0298 - val_acc: 0.9922


#### Test Predictions

In [11]:
saved_model = keras.models.load_model('model_output/weights-cnn-mnist.hdf5')
predictions = saved_model.predict_classes(X_test, verbose = 2)
np.std(history.history['loss'])

0.11277957157335788

#### Test Final Accuracy

In [12]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))
final_loss, final_acc = model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

10000/10000 [==============================] - 20s 2ms/step
Final loss: 0.0228, final accuracy: 0.9929
10000/10000 [==============================] - 20s 2ms/step
Final loss: 0.0228, final accuracy: 0.9929
